In [ ]:
!python -m venv .venv

In [2]:
!pwd

/home/chisphung/ALPR_Fisheye/notebooks


In [5]:
%cd model/yolo/

/home/chisphung/ALPR_Fisheye/model/yolo


In [7]:
!    pip install -e .

Obtaining file:///home/chisphung/ALPR_Fisheye/model/yolo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached matplotlib-3.10.6-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached torch-2.8.0-

In [8]:
import argparse
import os
import sys
#/home/chisphung/ALPR_Fisheye/model/yolo/ultralytics/models/yolo/model.py
from model.yolo.ultralytics.models.yolo.model import YOLO

In [23]:
YOLO('yolo11x.pt').train(data='../../dataset/dataset.yaml', epochs=1, imgsz=640, batch=1, name='yolov11x_alpr_det_100e', device='cpu')

#CHUA DOI ANCHOR SIZE

Ultralytics 8.3.202 🚀 Python-3.12.3 torch-2.8.0+cu128 CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../dataset/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11x_alpr_det_100e10, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pe

KeyboardInterrupt: 